## Trabalho Prático 1 - Algoritmos 2

### Nome: Augusto Maillo Queiroga de Figueiredo
### Matrícula: 2019006450

Para executar localmente rode a célula abaixo para garantir que os widgets estejam habilitados no notebook e em seguida recarregue a página (F5). 

In [1]:
!jupyter nbextension enable --user --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
""" Imports """
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider, Button
from IPython.display import display, clear_output
from ipywidgets import widgets

In [3]:
class Point:
    """ Auxiliar class to work with points 
    """
    def __init__(self, x,y):
       self.x = x
       self.y = y

    def __add__(self, B):
        return Point(self.x + B.x, self.y + B.y)

    def __sub__(self, B):
        return Point(self.x - B.x, self.y - B.y)

    def __mul__(self, B):
        return self.x*B.y - B.x*self.y
    
    def __str__(self):
        return "(%.2f, %.2f) " %(self.x, self.y)

    def __eq__(self, B):
        return self.x == B.x and self.y == B.y

    def __repr__(self):
        return "(%.2f, %.2f) " %(self.x, self.y)

    def __hash__(self):
        return hash((self.x, self.y))

In [4]:
def convert_point_list(points):
    """ Convert a points list into two lists, one for x values and other for y values
    """
    x = []
    y = []
    for p in points:
        x.append(p.x)
        y.append(p.y)
    return x,y

def convert_into_point_list(x_list, y_list):
    """ Convert two lists with x values and y values into point list
    """
    points = []
    for x,y in zip(x_list, y_list):
        points.append(Point(x, y))
    return points

def inTriangle(p, v1, v2, v3):
    """ Check if p is inside triangle made from v1, v2, v3
    """
    return (v1-p)*(v2-v1) <= 0 and (v2-p)*(v3-v2) <= 0 and (v3-p)*(v1-v3) <= 0

def isCounterClockWise(points):
    """ Check if points list is in counter clock wise order """
    min_y = points[0][1]
    min_idx = 0
    for i, e in enumerate(points):
        if e[1] < min_y:
            min_y = e[1]
            min_idx = i
        elif e[1] == min_y:
            if e[0] > points[min_idx][0]:
                min_y = e[1]
                min_idx = i
        
    a = points[min_idx]
    a = Point(a[0], a[1])
    b = points[(min_idx-1) % len(points)]
    b = Point(b[0], b[1])
    c = points[(min_idx+1) % len(points)]
    c = Point(c[0], c[1])
    prod = (a - b)*(c - a)
    return prod > 0

# Ear Clipper

In [5]:
class EarClipper:
    """ Computes polygon triangulation by ear clipping method
    """
    def __init__(self, xs, ys):
        """ Define common variables and get triangulation result
        """
        self._starting_points = convert_into_point_list(xs, ys)
        self.points = self._starting_points.copy()
        self._solution = []
        self.index = 0
        self.xs = xs
        self.ys = ys

        self.triangles = []
        while( len(self.points) > 3):
            self.ear_clipping_aux(
                self.index%len(self.points)
                )
        self.triangles.append([p for p in self.points])

    def isEarTip(self, index):
        """ Auxiliar method to check if a point is an ear tip
        """
        first_seg = (self.points[index]- self.points[(index-1)%len(self.points)])
        second_seg = (self.points[(index+1)%len(self.points)] - self.points[index]) 

        # if exists a left turn
        if ( first_seg * second_seg < 0 ):
            # check if there is a point inside the triangle
            v1 = self.points[(index-1)%len(self.points)]
            v2 = self.points[index]
            v3 = self.points[(index+1)%len(self.points)]
            nindex = index + 2
            while nindex % len(self.points) != (index-1)%len(self.points):
                if inTriangle(
                    p = self.points[nindex%len(self.points)], 
                    v1 = v1, 
                    v2 = v2, 
                    v3 = v3
                ):
                    return False
                nindex+=1
            self.triangles.append([v1, v2, v3])
            return True
        return False


    def ear_clipping_aux(self, idx):
        """ Controls points popping and solution list
        """
        if(self.isEarTip(idx)):
            self._solution.append(
                [
                    self.points[(idx-1)%len(self.points)], 
                     self.points[(idx+1)%len(self.points)]
                ] 
                )
            self.points.pop(idx)
        else:
            self._solution.append(None)
            self.index+=1

    def solution(self):
        """ Returns solution found 
        """
        return self._solution

    def build_plots(self):
        """ Build a plot step by step list """
        """ returns one list of lists of points, one of segments and another with
        messages detailing the algorithm step by step
        these results are for plotting purposes
        """

        """ Starts by original polygon """
        self.points = self._starting_points
        points_index = {
            pt : i for i,pt in enumerate(self._starting_points)
        }

        points = [(xi, yi, '.b') for xi,yi in zip(self.xs, self.ys)]
        segments = [] 
        for i in range(1, len(points)+1):
            p = points[i % len(points)]
            q = points[(i-1) % len(points)]
            segments.append(([p[0], q[0]], [p[1], q[1]], '-b'))
        segments = [segments]
        points = [points]
        messages = ["Initial polygon"]
        
        i = 0
        for s in self._solution:
            """ Indicates which point is the reference at the step """
            segments.append([])
            points.append([])
            points[-1] = points[-2].copy()
            pt_index = points_index[self.points[i % len(self.points)]]
            points[-1][pt_index] = (
                points[-1][pt_index][0], 
                points[-1][pt_index][1], 
                '.r'
            )
            segments[-1] = segments[-2].copy()
            messages.append('Analyzing vertex %s ...' % self.points[i % len(self.points)])

            """ index i is an ear tip """
            if s is not None:
                """ plot segment added """
                segments.append([])
                points.append([])
                points[-1] = points[-2].copy()
                segments[-1] = segments[-2].copy()
                segments[-1].append(([s[0].x, s[1].x] , [s[0].y, s[1].y], '-k'))
                messages.append('Vertex %s is an ear tip' % self.points[i % len(self.points)])
                self.points.pop(i % len(self.points))

            else:
                """if not, revert vertex color and add a message """
                segments.append([])
                points.append([])
                points[-1] = points[-2].copy()
                points[-1][pt_index] = (
                    points[-1][pt_index][0], 
                    points[-1][pt_index][1],
                    '.b'
                    )
                segments[-1] = segments[-2].copy()
                messages.append('Vertex %s is not an ear tip' % self.points[i % len(self.points)])
                i+=1

        """ Build a intermediate plot showing triangulation final result """
        points.append(points[-1].copy())
        points[-1] = [ (e[0], e[1], '.r') for e in points[-1]]
        segments.append(segments[-1].copy())
        segments[-1] = [ (e[0], e[1], '-k') for e in segments[-1]]
        messages.append("Final triangulation")

        return points, segments, messages

    def get_triangles(self):
        """ Return polygon triangulation """
        return self.triangles

In [6]:
class TrianglesTree:
    """ Graph with triangles to get 3-coloring """
    def __init__(self, triangles):
        """ Build adjacency matrix and start all vertex with same color at step 0 """
        self._triangles = triangles;
        self._adjacency_matrix = [[] for e in triangles]
        self._vertices = set()
        for e in triangles:
            for v in e:
                self._vertices.add(v)
        self.default_color = {
            v : 0 for v in self._vertices
        }

        self._vertices_colors = [self.default_color.copy()]

        for i in range(len(triangles)):
            for j in range(len(triangles)):
                if i == j:
                    continue
                s = set()
                for e in triangles[i]: s.add(e)
                for e in triangles[j]: s.add(e)
                
                """ If has two vertices in common should be neighbors """
                if len(s) == 4:
                    self._adjacency_matrix[i].append(j)

    def colour_triangle(self, t):
        """ Colour triangle vertexes """
        to_color = None
        s = 0
        for v in t:
            c = self._vertices_colors[-1][v]
            if c == 0:
                to_color = v
            s += c
        if to_color is not None:
            self._vertices_colors.append(self._vertices_colors[-1].copy())
            self._vertices_colors[-1][to_color] = 6 - s

    def dfs_aux(self, v, visiteds):
        """ auxiliar method for DFS """
        visiteds.add(v)
        for ngb in self._adjacency_matrix[v]:
            if ngb not in visiteds:
                self.colour_triangle(self._triangles[ngb])
                self.dfs_aux(ngb,visiteds)

    def dfs(self):
        """ DFS core. Starting coloring all vertex from first triangle
        with different colors 
        """
        visiteds = set()

        """ Start first vertex with 3 different colours """
        self._vertices_colors.append(self._vertices_colors[-1].copy())
        for i,v in enumerate(self._triangles[0]):
            self._vertices_colors[-1][v] = i + 1
        self.dfs_aux(0, visiteds)

        return self._vertices_colors

    def get_adj(self):
        """ Return adjacency matrix of triangles """
        return self._adjacency_matrix


In [7]:
%matplotlib inline
plt.ion()

class Index:
    """ Solution index control
    Work as a helper to navegate through solution steps
    """ 
    def __init__(self, max, callback):
        self.ind = -1
        self.max = max
        self.callback = callback

    def next(self, event):
        self.ind = min(self.ind + 1 , self.max)
        self.callback(self.ind)
        
    def prev(self, event):
        self.ind = max((self.ind - 1 ), 0)
        self.callback(self.ind)

    def first(self, event):
        self.ind = 0
        self.callback(self.ind)


    def last(self, event):
        self.ind = self.max
        self.callback(self.ind)


class Plotter:
    """ Plotting controller, builds buttons, messages and control step plotting
    """
    def __init__(self, points, segments, messages):

        self.pts = points
        self.segs = segments
        self.messages = messages

        self.button1 = widgets.Button(
            description="Next step", 
            layout = widgets.Layout(width="200px", height='60px')
            )
        self.button2 = widgets.Button(
            description="Previous step", 
            layout = widgets.Layout(width="200px", height='60px')
            )
        self.button3 = widgets.Button(
            description="First step", 
            layout = widgets.Layout(width="200px", height='60px')
            )
        self.button4 = widgets.Button(
            description="Last step", 
            layout = widgets.Layout(width="200px", height='60px')
            )
        self.text_box  = widgets.HTML()
        self.out = widgets.Output()
#         self.plt_aux(0)

        buttons = widgets.HBox(children=[self.button3, self.button2, self.button1, self.button4])
        buttons.layout.align_items='stretch'

        column1 =  widgets.VBox(children=[self.out, buttons])
        all_widgets = widgets.VBox(children=[self.text_box, column1])
        all_widgets.layout.align_items='stretch'
        plt.close()

        display(all_widgets)
        
#         clear_output(wait=True)


        """ Define callbacks for button clicks """
        callback = Index(len(self.pts)-1, self.plt_aux)
        self.button1.on_click(callback.next)
        self.button2.on_click(callback.prev)
        self.button3.on_click(callback.first)
        self.button4.on_click(callback.last)

        
        callback.next(None)
    def plt_aux(self, index):
        """ Given an index plots a solution state 
        """
        with self.out:
            if index == 0:
                self.button2.disabled=True
            else:
                self.button2.disabled=False
            if index == len(self.messages) -1:
                self.button1.disabled=True
            else:
                self.button1.disabled=False

            self.fig = plt.figure(figsize=(6,4), dpi=160)
            plt.title("Ear Clipping Step by Step")
            to_plot_pts = self.pts[index]
            to_plot_segs = self.segs[index]
            for seg in to_plot_segs:
                plt.plot(seg[0], seg[1], seg[2],)
            for pt in to_plot_pts:
                plt.plot(pt[0], pt[1], pt[2], markersize=15)
            plt.show()
            self.text_box.value = f"<p style='font-size:24px'><b><font color='black'>{self.messages[index]}</b></p>"
            clear_output(wait=True)


In [8]:
def solution_wrapper(points):
    if isCounterClockWise(points):
        points = points[::-1]
    xs = [e[0] for e in points]
    ys = [e[1] for e in points]

    ear_clipper = EarClipper(xs, ys)
    # Get algorithm steps
    pts, segs, messages = ear_clipper.build_plots()


    triangles = ear_clipper.triangles
    three_color_result = TrianglesTree(triangles).dfs()

    three_color_points = []
    three_color_segments = []

    color_map ={0 : '.b', 1: '.k', 2: '.y', 3: '.m'}

    """ Generating list of points and its colors for 3-coloring step """
    for d in three_color_result:
        pts_list = []
        for e in pts[-1]:
            pts_list.append((e[0], e[1], color_map[d[Point(e[0], e[1])]]))
        pts.append(pts_list)
        segs.append(segs[-1].copy())
        messages.append("Coloring...")
        qt = {1: 0, 2: 0, 3: 0 }
        d = three_color_result[-1]
        for k,v in d.items():
            qt[v]+=1
    messages[-1] = "Finished. The minimum number of cameras in this galery is %d" % min(qt.values())
    Plotter(pts, segs, messages)

## Test 1

In [9]:
input_ = [(0,5), (2,6), (4,4), (5,8), (7,3), (6,1), (4, 1.5), (1,0)]
solution_wrapper(input_)

## Test 2

In [10]:
input_ =  [(0, 0),
 (0.5, 10),
 (1.0, 2),
 (1.5, 2),
 (2.0, 10),
 (2.5, 2),
 (3.0, 2),
 (3.5, 10),
 (4.0, 2),
 (4.5, 2),
 (5.0, 10),
 (5.5, 2),
 (6.0, 2),
 (6.5, 10),
 (7.0, 2),
 (7.5, 2),
 (8.0, 10),
 (8.5, 2),
 (9.0, 2),
 (9.5, 10),
 (10.0, 2),
 (10.5, 2),
 (11.0, 10),
 (12, 0),
]
solution_wrapper(input_)

## Test 3

In [11]:
input_ = [(0,0), (1,3), (4,-2),(6.5,4),(2,4.5),(7.5,7),(1.5,7.5),(0.6,5),(-0.8,6)]
solution_wrapper(input_)

## Test 4

In [12]:
input_ = [(0.7071067811865476, 0.7071067811865475),
 (6.123233995736766e-17, 1.0),
 (-0.7071067811865475, 0.7071067811865476),
 (-1.0, 1.2246467991473532e-16),
 (-0.7071067811865477, -0.7071067811865475),
 (-1.8369701987210297e-16, -1.0),
 (0.7071067811865475, -0.7071067811865477),
 (1.0, -2.4492935982947064e-16)]


solution_wrapper(input_)

## Test 5

In [13]:
input_ = [(3,4), 
(2,2), 
(3.5025,1.02125), 
(3.8025,2.64125), 
(4.7825,1.22125), 
(6.2225,1.20125), 
(6.5225,2.42125), 
(5.6025,3.58125), 
(5.0625,2.52125), 
(4.3425,3.48125), 
(5.3825,4.68125)]


solution_wrapper(input_)

## Test 6

In [14]:
input_ = [(-4,0), (-3,1), (-3, 10), (-1, 6), (0, 6), (2,4), (3,5), (4,0),(1,2), (0,-2)]
solution_wrapper(input_)